In [2]:
#NOTICE: This code follows the tutorial from TensorFlow on generating text
# They're very cool. Here's their license:
#
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [3]:
# dependencies
import tensorflow as tf
import numpy as np
import os

In [4]:
corpus = open('data.txt', encoding='utf-8').read()

In [5]:
# print first 250 characters juuuust to make sure
print(corpus[:250])

Academics
Research
The gradual ramp-up of in-person research is continuing.
Global: For fall 2020, "Go Local" is a "go", Study Away is a "no" (mostly)
Owing to ongoing international travel restrictions and related health complexities, the University 


In [6]:
# unique characters in file
vocab = sorted(set(corpus))
print ('{} unique characters'.format(len(vocab)))

84 unique characters


In [7]:
# map characters to indices and vice versa for feeding into LSTM and interpreting afterward
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in corpus])

In [8]:
# show how the first 13 characters from the text are mapped to integers juuuust to make sure
print ('{} ---- characters mapped to int ---- > {}'.format(repr(corpus[:13]), text_as_int[:13]))

'Academics\nRes' ---- characters mapped to int ---- > [28 55 53 56 57 65 61 55 71  0 44 57 71]


In [9]:
# maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(corpus)//(seq_length+1)

# create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

A
c
a
d
e


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Academics\nResearch\nThe gradual ramp-up of in-person research is continuing.\nGlobal: For fall 2020, "G'
'o Local" is a "go", Study Away is a "no" (mostly)\nOwing to ongoing international travel restrictions '
'and related health complexities, the University has notified students enrolled in Study Away for fall'
' 2020 that it is limiting participation to students who do not require issuance of a student visa or '
'who are already present in the host country or region where they were intending to study. \n\nIn concer'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
# batch size
BATCH_SIZE = 32

# buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements). Thanks TF!
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

In [13]:
# length of the vocabulary in chars
vocab_size = len(vocab)

# embedding dimension
embedding_dim = 256

# number of RNN units
rnn_units = 1024

# number of epochs
EPOCHS = 75

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [15]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           21504     
_________________________________________________________________
gru (GRU)                    (32, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (32, None, 84)            86100     
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [16]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [17]:
model.compile(optimizer='adam', loss=loss)

In [18]:
# directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# regular callback
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

callbacks_list = [checkpoint_callback, callback]

In [19]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=callbacks_list)

Epoch 1/75
12/12 [==============================] - 28s 2s/step - loss: 4.3515
Epoch 2/75
12/12 [==============================] - 36s 3s/step - loss: 3.5774
Epoch 3/75
12/12 [==============================] - 40s 3s/step - loss: 3.0264
Epoch 4/75
12/12 [==============================] - 86s 7s/step - loss: 2.8357
Epoch 5/75
12/12 [==============================] - 39s 3s/step - loss: 2.6559
Epoch 6/75
12/12 [==============================] - 33s 3s/step - loss: 2.5309
Epoch 7/75
12/12 [==============================] - 44s 4s/step - loss: 2.4456
Epoch 8/75
12/12 [==============================] - 34s 3s/step - loss: 2.3779
Epoch 9/75
12/12 [==============================] - 37s 3s/step - loss: 2.3172
Epoch 10/75
12/12 [==============================] - 30s 2s/step - loss: 2.2607
Epoch 11/75
12/12 [==============================] - 32s 3s/step - loss: 2.2046
Epoch 12/75
12/12 [==============================] - 31s 3s/step - loss: 2.1470
Epoch 13/75
12/12 [==============================

In [20]:
# rebuild model with latest checkpoints
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            21504     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86100     
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [21]:
def generate_text(model, start_string):
  # evaluation step (generating text using the learned model)

  # number of characters to generate
  num_generate = 1000

  # converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # empty string to store our results
  text_generated = []

  # low temperatures results in more predictable text.
  # higher temperatures results in more surprising text.
  # :D
  temperature = 1.0

  # here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [22]:
print(generate_text(model, start_string='The '))

The University. However, attenden by the coronavirus, or who have family members or loved ones or friends stricken by For of the impace of the Summer.
The College of Dentistry Resumes Providing Thoug on a day of pride for d July 6 — will be held remotely; administrative personnel with the option of arranging to come to residence halls to retrive the semester in the coming been exsended to main access to their office for recent research to in oncemelty advicorvort the a restrictions in privary to combat the spread of COVID-19.


Academic Ye is virtually Unter pack up thein possessions of New York State mandates. This will continue to communicate about our plonsent a some are moint, and to continue to health effort to assist students who have financial need caused by COVID-19, in the layt through the end of June. Once the trajectory of COVID-19, NYU has made rooms where students packed up the reach of COVID-19; the NYU community will be the belongings back to the increased research activ

In [26]:
!mkdir -p saved_model
model.save('saved_model/my_model') 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets
